In [1]:
import networkx as nx
import random
import copy
import time
import matplotlib.pyplot as plt
import sys
from collections import defaultdict
%matplotlib tk
initial_start = time.time()

NUM_NODES = 2000
G = nx.barabasi_albert_graph(NUM_NODES, 3)

#G = nx.watts_strogatz_graph(NUM_NODES, 6, 1)

In [2]:
''' Graph Characterization Metrics '''
print("Average degree = ", 2 * G.number_of_edges() / G.number_of_nodes())
print("Clustering Coefficient = ", nx.average_clustering(G))
print("Average Path Length = ", nx.average_shortest_path_length(G))

Average degree =  5.991
Clustering Coefficient =  0.021069338148557187
Average Path Length =  3.704672336168084


In [3]:
# def kawai_draw(G, infected=[]) :
#     ''' Draw the graph 'G' following the kamada-kawai algorithm
#         and coloring the nodes on the 'infected' variable as red
#         and the others as green
#     '''
#     colors = []
#     for node in G :
#         if node in infected :
#             colors.append('red')
#         else :
#             colors.append('green')
#     nx.draw(G, layout=nx.kamada_kawai_layout(G), node_color=colors, node_size = 30, alpha = 0.7)
#     plt.show()
    
# kawai_draw(G)
# plt.show()

In [4]:
start = time.time()

spl = nx.all_pairs_shortest_path_length(G)

def change_ds(spl_gen, num_nodes, max_radius):
    ''' 
    Transforms 'spl_gen' into the accepted datastructure
    by the 'infected_average_by_separation_degree' method
    '''
    result = []
    for n in range(num_nodes):
        di_tmp = {}
        for entry in spl_gen:  
            for radius in range(1, max_radius + 1):
                li_tmp = []
                for node, dist in entry[1].items():
                    if dist == radius:
                        li_tmp.append(node)
                if li_tmp != []:
                    di_tmp[radius - 1] = li_tmp
            result.append(copy.copy(di_tmp))
            
    return result

MAX_RADIUS_TO_CHECK = 6
separation_degrees = change_ds(spl, NUM_NODES, MAX_RADIUS_TO_CHECK)

print('Time:', time.time() - start, "seconds")

Time: 16.23072338104248 seconds


In [5]:
def infected_percentage(nodes, infected) :
    count = 0
    for node in nodes :
        if node in infected :
            count += 1
    
    return count / len(nodes)

def infected_average_by_separation_degree(G, separation_degrees, infected) :
    node_count = {}
    percentage_sum = {}
    
    for node in range(0, len(separation_degrees)) :
        if node in infected :
            for degree in separation_degrees[node] :
                if degree not in percentage_sum.keys() :
                    percentage_sum[degree] = 0
                    node_count[degree] = 0
                percentage_sum[degree] += infected_percentage(separation_degrees[node][degree], infected)
                node_count[degree] += 1
        
    average_by_separation_degree = {}
    
    for separation_degree in percentage_sum :
        average_by_separation_degree[separation_degree] = percentage_sum[separation_degree] / node_count[separation_degree]
            
    return average_by_separation_degree

In [6]:
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.SIModel as si
import ndlib.models.opinions.VoterModel as vt

# Model selection
model = si.SIModel(G)
#model = vt.VoterModel(G)

# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.05) # probability of infection
cfg.add_model_parameter("percentage_infected", 0) # initially infected % (if 0 => 1 infected)
model.set_initial_status(cfg)

C:\ProgramData\Anaconda3\lib\site-packages\ndlib\models\DiffusionModel.py:161: UserWarning: Graph with less than 100 nodes: a single node will be set as infected
  warnings.warn('Graph with less than 100 nodes: a single node will be set as infected')


In [7]:
start = time.time()

infected = []
it_count = 0
while len(infected) < NUM_NODES / 2 :
    it_count += 1
    iteration = model.iteration()
    for node, i in iteration['status'].items() :
        if i == 1:
            infected.append(node)

    
avg_traits = infected_average_by_separation_degree(G, separation_degrees, infected)
infected_count = len(infected)

print('Infected count', infected_count)
print('Iterations', it_count)
print(avg_traits)

print('Time:', time.time() - start, "seconds")

Infected count 1035
Iterations 35
{0: 0.7326842971614199, 1: 0.6633015692802294, 2: 0.6032980675028664, 3: 0.46361330920899707, 4: 0.2778716075921198, 5: 0.07839781146410596}
Time: 18.714564561843872 seconds


In [8]:
start = time.time()

''' Random distribution of infected nodes '''
def simulate_random_infecteds(G_rand, num_nodes, radius_split_rand, num_infects):
    '''
    Randomly picks the same number of infecteds as in the 
    non-random model (for each iteration of the spreading)
    Returns a list of @infected_average_by_radius lists
    '''
    random_infected = []
    random_not_infected = list(range(0, num_nodes))
    
    for _ in range(num_infects):
        new_inf = random.choice(random_not_infected)
        random_infected.append(new_inf)
        random_not_infected.remove(new_inf)

    avg_trait_sharing_by_radius = infected_average_by_separation_degree(G_rand, separation_degrees, random_infected)
    print(avg_trait_sharing_by_radius)    

    return (avg_trait_sharing_by_radius, random_infected)

avg_random_trait_sharing_by_radius, final_random_infected = simulate_random_infecteds(G, NUM_NODES, separation_degrees, infected_count)

print('Time:', time.time() - start, "seconds")

{0: 0.5163219344849977, 1: 0.5349391151897711, 2: 0.5238904986376355, 3: 0.5140210305312011, 4: 0.5490007373139736, 5: 0.613181770786525}
Time: 22.882129430770874 seconds


In [9]:
def propensity_by_radius(radius, avg_traits, rand_avg_traits):
    return (avg_traits[radius-1] / rand_avg_traits[radius-1]) - 1

def all_propensities(max_radius, avg_traits, rand_avg_traits):
    all_propensities = []
    for r in range(1, max_radius + 1):
        all_propensities.append(propensity_by_radius(r, avg_traits, rand_avg_traits))
    return all_propensities

all_props = all_propensities(MAX_RADIUS_TO_CHECK, avg_traits, avg_random_trait_sharing_by_radius)

def normalized_propensities(max_radius, all_propensities):
    normalized_propensities = []
    for r in range(max_radius):
        normalized_propensities.append(all_propensities[r] / all_propensities[0])
    
    return normalized_propensities
        
print(normalized_propensities(MAX_RADIUS_TO_CHECK, all_props))

[1.0, 0.5726279161132591, 0.3617097685600657, -0.2340211114193963, -1.1785339894092535, -2.0812679521266007]


In [10]:
''' Check maximum degree of separation. '''
max_sep_deg = -1
for node in separation_degrees:
    if max_sep_deg < len(node):
        max_sep_deg = len(node)
print("Max separation degree: ", max_sep_deg)

''' Count total nodes per degree of separation. '''
total_nodes_per_sep_deg = [0 for _ in range(max_sep_deg)]
for node in separation_degrees :
    for dist in node:
        total_nodes_per_sep_deg[dist] += len(node[dist])

''' Calculate average nodes per degree of separation.
    Note that values can be < 0 because not every node
    has the same number of maximum separation degrees. '''
avg_nodes_per_sep_deg = []
for total_nodes in total_nodes_per_sep_deg:
    avg_nodes_per_sep_deg.append(total_nodes/len(separation_degrees))

print("Avg nodes per degree of separation:", avg_nodes_per_sep_deg)

print("Total time:", time.time() - initial_start)

Max separation degree:  6
Avg nodes per degree of separation: [5.991, 86.995, 619.868, 1065.928, 219.2255, 3.5385]
Total time: 74.27696919441223


In [11]:
# Saving graph to file
'''
import pickle

with open('2000_nodes_scale_free.graph', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(G, f)
'''

"\nimport pickle\n\nwith open('2000_nodes_scale_free.graph', 'wb') as f:  # Python 3: open(..., 'wb')\n    pickle.dump(G, f)\n"

In [12]:
# Loading graph from file
'''
import pickle

with open('2000_nodes_scale_free.graph', 'rb') as f:  # Python 3: open(..., 'rb')
    Gtest = pickle.load(f)

print(2 * Gtest.number_of_edges() / Gtest.number_of_nodes())
'''

"\nimport pickle\n\nwith open('2000_nodes_scale_free.graph', 'rb') as f:  # Python 3: open(..., 'rb')\n    Gtest = pickle.load(f)\n\nprint(2 * Gtest.number_of_edges() / Gtest.number_of_nodes())\n"